- Ranking of the Korean Actors/Actresses on the [cine21](https://www.cine21.com/rank/person)
- cine21에서는 url이 변하지 않음. 
- 개발자모드 - network
- 보통 클릭에 따라 주소가 바뀌면 GET METHOD, 그게 아니면 보통 POST METHOD를 사용한다. 
- POST METHOD에서는 Form Data에 담겨서 전달되고, GET방식에서는 주소에 보통 같이 전달된다. 

The information we need 
- Request URL: http://www.cine21.com/rank/person/content
- Request Method : POST
- Format Data of POST:
    - secion : actor
    - period_start : '2019-02'(changed with period button in the web page)
    - gender = 'all'(changed with gender button in the web page)
    - page = 1 (changed with pagination button below)

### 1. Import Library

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo

import re

### 2. mongodb connection

In [2]:
conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

### 3. crawling address requests

In [3]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2020-07'
post_data['gender'] = 'all'
post_data['page'] = 1

In [4]:
res = requests.post(cine21_url, data=post_data)

### 4. parsing & extracting data
- regex reference : https://www.fun-coding.org/DS&AL3-4.html

In [5]:
soup = BeautifulSoup(res.content, 'html.parser')

In [7]:
actors = soup.select('li.people_li div.name')
for actor in actors:
    print(re.sub('\(\w*\)', '', actor.text))

황정민
이정재
조우진
강동원
이정현
박정민
성동일


### 5. extracting the detail information of actor/actresses
- need to click the actor
- need to know the address of the actor detail page first

reference: speacial regex
```html
    - Greedy(.*) : <.*> 최대치를 다 가져옴. 
        vs Non-Greedy(.*?) : ?를 넣으면 첫번째 매칭이 되는 데 까지만 패턴으로 인지. 
    - https://regexr.com
    - . 문자는 줄바꿈 문자인 \n을 제외한 모든 문자 한개를 의미함. 
    - *는 문자가 0번 또는 그 이상 반복되는 패턴을 의미함. 
    - .* 는 한 문자가 0번 또는 그 이상 반복되는 패턴을 의미. 
```


In [ ]:
#

In [52]:
actors = soup.select('li.people_li div.name')

actors_info_list = list()


for actor in actors:
    actor_link = 'http://www.cine21.com/'+actor.select_one('a').attrs['href']
    print(actor_link)
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser')

    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')

    actor_info_dict = dict()
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value)
        actor_info_dict[actor_item_field] = actor_item_value.strip()

    actors_info_list.append(actor_info_dict)


print(actors_info_list)

http://www.cine21.com//db/person/info/?person_id=14268
http://www.cine21.com//db/person/info/?person_id=1075
http://www.cine21.com//db/person/info/?person_id=98611
http://www.cine21.com//db/person/info/?person_id=18040
http://www.cine21.com//db/person/info/?person_id=2760
http://www.cine21.com//db/person/info/?person_id=74826
http://www.cine21.com//db/person/info/?person_id=27826
[{'직업': '배우', '생년월일': '1970-09-01', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과 졸업', '특기': '농구, 악기연주', '소속사': '예당엔터테인먼트'}, {'직업': '배우', '성별': '남', '홈페이지': 'http://www.artistcompany.co.kr/portfolio-item/lee-jung-jae/', '소속사': '아티스트컴퍼니'}, {'다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'}, {'직업': '배우', '생년월일': '1981-01-18', '성별': '남', '홈페이지': 'http://www.gangdongwon.com', '신장/체중': '186cm, 68kg', '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)', '취미': '음악감상, 웹서핑, 운동, 신발 모으기', '특기': '축구'}, {'직업': '배우', '생년월일': '1980-02-07', '성별': '여', '홈페이지': 'https://www.instagram.com/mermaidleejh/\nhttps://twitter.com/m

### 6. extracting index & movies list

In [73]:
actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')

for index, actor in enumerate(actors):
    print(re.sub('\(\w*\)', '', actor.text))
    print(int(hits[index].text.replace(',', '')))
    movie_titles = movies[index].select('li a span')
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
        
    print(movie_title_list)

황정민
90206
['다만 악에서 구하소서', '와이키키 브라더스', '신세계', '국제시장']
이정재
60153
['다만 악에서 구하소서', '신과 함께-죄와 벌', '태양은 없다', '시월애', '신과 함께-인과 연', '신세계']
조우진
34691
['발신제한', '도굴', '서복', '국가부도의 날', '강철비', '1987']
강동원
33732
['반도', '1987']
이정현
30695
['반도', '죽지않는 인간들의 밤']
박정민
30074
['다만 악에서 구하소서', '동주', '시동', '파수꾼', '그것만이 내 세상']
성동일
26594
['담보', '수상한 그녀']


### 7. making dictionary 

In [82]:
actors_info_list = list()
actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')
# rankings = soup.select('li.people_li > span.grade')

for index, actor in enumerate(actors):
    
#     print(rankings[index].text)

    
    actor_name = re.sub('\(\w*\)', '', actor.text)
    actor_hits = int(hits[index].text.replace(',', ''))
    movie_titles = movies[index].select('li a span')
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
          
    actor_info_dict = dict()    
    actor_info_dict["배우이름"]  = actor_name
    actor_info_dict["흥행지수"]  = actor_hits
    actor_info_dict["출연영화"]  = movie_title_list
    
    actor_link = 'http://www.cine21.com/'+actor.select_one('a').attrs['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value)
        actor_info_dict[actor_item_field] = actor_item_value.strip()

    actors_info_list.append(actor_info_dict)


print(actors_info_list)

1
2
3
4
5
6
7
[{'배우이름': '황정민', '흥행지수': 90206, '출연영화': ['다만 악에서 구하소서', '와이키키 브라더스', '신세계', '국제시장'], '직업': '배우', '생년월일': '1970-09-01', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과 졸업', '특기': '농구, 악기연주', '소속사': '예당엔터테인먼트'}, {'배우이름': '이정재', '흥행지수': 60153, '출연영화': ['다만 악에서 구하소서', '신과 함께-죄와 벌', '태양은 없다', '시월애', '신과 함께-인과 연', '신세계'], '직업': '배우', '성별': '남', '홈페이지': 'http://www.artistcompany.co.kr/portfolio-item/lee-jung-jae/', '소속사': '아티스트컴퍼니'}, {'배우이름': '조우진', '흥행지수': 34691, '출연영화': ['발신제한', '도굴', '서복', '국가부도의 날', '강철비', '1987'], '다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'}, {'배우이름': '강동원', '흥행지수': 33732, '출연영화': ['반도', '1987'], '직업': '배우', '생년월일': '1981-01-18', '성별': '남', '홈페이지': 'http://www.gangdongwon.com', '신장/체중': '186cm, 68kg', '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)', '취미': '음악감상, 웹서핑, 운동, 신발 모으기', '특기': '축구'}, {'배우이름': '이정현', '흥행지수': 30695, '출연영화': ['반도', '죽지않는 인간들의 밤'], '직업': '배우', '생년월일': '1980-02-07', '성별': '여', '홈페이지': 'https://www.instagram.com/mermaidleejh/\

### 8. Pagination Crwaling
- change the Form Data on the POST REQUEST

In [86]:
from bs4 import BeautifulSoup
import requests
import pymongo

import re

actors_info_list = list()

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2020-07'
post_data['gender'] = 'all'
post_data['page'] = 1


for index in range(1, 21):
    post_data['page'] = index
    
    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')


    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade')


    
    for index, actor in enumerate(actors):

        actor_name = re.sub('\(\w*\)', '', actor.text)
        actor_hits = int(hits[index].text.replace(',', ''))
        movie_titles = movies[index].select('li a span')
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)

        actor_info_dict = dict()    
        actor_info_dict["배우이름"]  = actor_name
        actor_info_dict["흥행지수"]  = actor_hits
        actor_info_dict["출연영화"]  = movie_title_list
        actor_info_dict["랭킹"] = rankings[index].text
        
        actor_link = 'http://www.cine21.com/'+actor.select_one('a').attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')
        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value)
            actor_info_dict[actor_item_field] = actor_item_value.strip()

        actors_info_list.append(actor_info_dict)


print(actors_info_list)

[{'배우이름': '황정민', '흥행지수': 90206, '출연영화': ['다만 악에서 구하소서', '와이키키 브라더스', '신세계', '국제시장'], '랭킹': '1', '직업': '배우', '생년월일': '1970-09-01', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과 졸업', '특기': '농구, 악기연주', '소속사': '예당엔터테인먼트'}, {'배우이름': '이정재', '흥행지수': 60153, '출연영화': ['다만 악에서 구하소서', '신과 함께-죄와 벌', '태양은 없다', '시월애', '신과 함께-인과 연', '신세계'], '랭킹': '2', '직업': '배우', '성별': '남', '홈페이지': 'http://www.artistcompany.co.kr/portfolio-item/lee-jung-jae/', '소속사': '아티스트컴퍼니'}, {'배우이름': '조우진', '흥행지수': 34691, '출연영화': ['발신제한', '도굴', '서복', '국가부도의 날', '강철비', '1987'], '랭킹': '3', '다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'}, {'배우이름': '강동원', '흥행지수': 33732, '출연영화': ['반도', '1987'], '랭킹': '4', '직업': '배우', '생년월일': '1981-01-18', '성별': '남', '홈페이지': 'http://www.gangdongwon.com', '신장/체중': '186cm, 68kg', '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)', '취미': '음악감상, 웹서핑, 운동, 신발 모으기', '특기': '축구'}, {'배우이름': '이정현', '흥행지수': 30695, '출연영화': ['반도', '죽지않는 인간들의 밤'], '랭킹': '5', '직업': '배우', '생년월일': '1980-02-07', '성별': '여', '홈페이지': 

### 9. Input Data to Mongo DB

In [87]:
conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

In [88]:
actor_collection.insert_many(actors_info_list)

In [91]:
actor_collection.delete_many({})

### 10. final version

In [92]:
from bs4 import BeautifulSoup
import requests
import pymongo

import re


conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection


actors_info_list = list()

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2020-07'
post_data['gender'] = 'all'
post_data['page'] = 1


for index in range(1, 21):
    post_data['page'] = index
    
    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')


    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade')


    
    for index, actor in enumerate(actors):

        actor_name = re.sub('\(\w*\)', '', actor.text)
        actor_hits = int(hits[index].text.replace(',', ''))
        movie_titles = movies[index].select('li a span')
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)

        actor_info_dict = dict()    
        actor_info_dict["배우이름"]  = actor_name
        actor_info_dict["흥행지수"]  = actor_hits
        actor_info_dict["출연영화"]  = movie_title_list
        actor_info_dict["랭킹"] = rankings[index].text
        
        actor_link = 'http://www.cine21.com/'+actor.select_one('a').attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')
        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value)
            actor_info_dict[actor_item_field] = actor_item_value.strip()

        actors_info_list.append(actor_info_dict)


actor_collection.insert_many(actors_info_list)